In [42]:
!pip install psaw
!pip install jsonlines

In [43]:
!pip install pandas

In [44]:
!pip install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [45]:
import requests
import pandas as pd
import re
import datetime as dt

In [46]:
def get_data(url):
    a = requests.get(url)
    return a.json()

In [47]:
import os
DATA_ROOT = '../data/'

from psaw import PushshiftAPI
api = PushshiftAPI()

In [48]:
#date to fetch submissions after
start_epoch=int(dt.datetime(2018, 1, 1).timestamp())

In [79]:
subreddit_arr = ['ExplainLikeImFive', 'FoodForThought', 'ChangeMyView', 'TodayILearned']
submission_filter = ['id', 'subreddit', 'url', 'title', 'selftext', 'author', 'score', 'num_comments','after']
comment_filter = ['id', 'subreddit', 'permalink', 'body', 'author', 'parent_id', 'title', 'num_comments','score']
# id Id of the comment string
# subreddit Name of the subreddit string
# full_link Full link of the comment string
# body The body of the comment text_en
# author The original poster of the comment string
# parent_id Id of the comment/submission for which this
# comment is made.
# string
# parent_body The body of the parent comment/submission text_en Y
# is_submission A flag denoted is this record a submission or not boolean Y
# created_at Convert the created_utc to datetime format
# (YYYY-MM-DDThh:mm:ssZ)
# pdate Y

In [80]:
def getSubmission(items, subreddit_name = "Any", topic="None"):
    if subreddit_name == "Any" and topic != None:
        gen = api.search_submissions(
            filter=submission_filter, # list of fields to return
            after = start_epoch,
            q = topic,
            limit = items, # limit on the number of records returned
            num_comments=">10",
            selftextnot = "[removed]"
        )
    else:
        gen = api.search_submissions(
            subreddit=[subreddit_name], # one or more subreddits to include in the search
            filter=submission_filter, # list of fields to return
            limit = items, # limit on the number of records returned
            num_comments=">10",
            after = start_epoch,
            selftextnot = "[removed]"
        )
    return gen

In [151]:
def addFieldsSub(df,topic="None"):
# 1st fied: add topic
    topic_field = [topic]*len(df)
    df['topic'] = topic_field

# 2nd field: add is_sub
    isSub = [True]*len(df)
    df['full_link'] = df['url']
    df['isSubmission'] = isSub
    
# 3rd field: convert created_utc to human readable
    df['created_at'] = pd.to_datetime(df['created_utc'], utc=True, unit='s')
    df['created_at'] = df.created_at.dt.strftime('%d/%m/%y %H:%M:%S')
    
    df.drop('created_utc', axis=1, inplace=True)
    df.drop('created', axis=1, inplace=True)
    df.drop('num_comments', axis=1, inplace=True)
    df.drop('score', axis=1, inplace=True)
    df.drop('d_', axis=1, inplace=True)
    df.drop('url', axis=1, inplace=True)
    
    coms = []
    for index in range(len(df)):
        if (("[removed]" == df["selftext"].iloc[index]) or ("[deleted]" == df["selftext"].iloc[index])):
            coms.append(index)
    df.drop(labels=coms, axis=0, inplace=True)
    df_final = df.reset_index(drop=True)
    return df_final

In [130]:
def addFieldsCom(df, df1):
# 1st fied: add add parent body
    df = pd.DataFrame(df)
    df['parent_body'] = ["dummy"]*len(df)
    print(type(df))
    i = 0
    for index in range(len(df)):
        key = df['parent_id'].iloc[index][3:]
        df['parent_body'].iloc[index] = df1['selftext'].loc[df1['id']==key]
        #print(df['parent_body'].iloc[index])
        #df['parent_body'].iloc[index] = df['parent_body'].iloc[index].value.tolist()[0]
    
# 3rd field: convert created_utc to human readable
    df['created_at'] = pd.to_datetime(df['created_utc'], utc=True, unit='s')
    df['created_at'] = df.created_at.dt.strftime('%d/%m/%y %H:%M:%S')
    
# 4th field: Full link
    df['prefix'] = ["https//:reddit.com"]*len(df)
    df['full_link'] = df['prefix'] + df['permalink']
    
#  Drop fields
    df.drop('created_utc', axis=1, inplace=True)
    df.drop('created', axis=1, inplace=True)
    df.drop('score', axis=1, inplace=True)
    df.drop('d_', axis=1, inplace=True)
    df.drop('prefix', axis=1, inplace=True)
    df.drop('permalink', axis=1, inplace=True)
    
    return df

In [152]:
submission_subreddit1_df = pd.DataFrame(list(getSubmission(300,'ExplainLikeImFive')))
submission_subreddit1_df = addFieldsSub(submission_subreddit1_df)
submission_subreddit1_df

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,Addy_Bishop,ubrebd,I understand what stock is but who creates it?...,explainlikeimfive,ELI5 where does stock come from?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,25/04/22 18:07:22
1,Ambitious_Category85,trsva6,,explainlikeimfive,ELI5: How do seasons exists? Isn't the earth's...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 23:31:10
2,twerkinmidget,trs358,"Switching the clocks seems excessive, it IS th...",explainlikeimfive,ELI5: I don’t understand the point of time zon...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:52:06
3,AlienRouge,trrkza,,explainlikeimfive,ELI5: what is Roko’s Basilisk?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:31:22
4,Not-Alpharious,trpcdk,Wouldn’t it be more of a survival advantage to...,explainlikeimfive,ELI5: Why do animals have mating seasons inste...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:07:51
...,...,...,...,...,...,...,...,...,...
281,reapingsulls123,t8kxc7,Yes I know you have to legally get out of thei...,explainlikeimfive,ELI5: How emergency services manage to get to ...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 08:49:53
282,EazyAB,t8krqd,What exactly makes an atomic bomb so catastrop...,explainlikeimfive,ELI5: Atomic Bombs?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 08:38:17
283,beepingview,t8jfpd,"1. If you buy a smart TV, can you watch both N...",explainlikeimfive,Eli5: Can you watch both Network and Cable on ...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 07:04:35
284,Patdick-Funny,t8isyt,,explainlikeimfive,Eli5: What causes babies/children able to lear...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 06:21:02


In [153]:
submission_subreddit2_df = pd.DataFrame(list(getSubmission(300,'FoodForThought')))
submission_subreddit2_df = addFieldsSub(submission_subreddit2_df)
submission_subreddit2_df

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,stockhackerDFW,tbebwz,,Foodforthought,‘Limited’ Tactical Nuclear Weapons Would Be Ca...,None,https://www.scientificamerican.com/article/lim...,True,11/03/22 01:17:18
1,speckz,tays0a,,Foodforthought,"In the Ukraine Conflict, Fake Fact-Checks Are ...",None,https://www.propublica.org/article/in-the-ukra...,True,10/03/22 13:34:50
2,eddytony96,tahm50,,Foodforthought,Hour-long YouTube video essays are thriving in...,None,https://www.vox.com/the-goods/22967496/youtube...,True,09/03/22 21:06:08
3,speckz,ta7jv0,,Foodforthought,How I learned the horrifying truth about my bi...,None,https://torontolife.com/memoir/the-horrifying-...,True,09/03/22 13:15:57
4,tigrayadey,t9jmoj,,Foodforthought,U.N. rights chief says air strikes have killed...,None,https://www.reuters.com/world/africa/more-than...,True,08/03/22 15:45:45
...,...,...,...,...,...,...,...,...,...
295,RememberRossetti,p7p6wh,,Foodforthought,Afghanistan: A Nation Without Leadership,None,https://geopoliticstoday.substack.com/p/afghan...,True,19/08/21 22:27:07
296,taptapper,p7612f,,Foodforthought,Lay off Joe Biden: He didn’t 'lose' Afghanista...,None,https://www.rawstory.com/biden-afghanistan/,True,19/08/21 02:33:01
297,Questioned_answers,p74qf2,,Foodforthought,The 13th amendment did not end slavery in Amer...,None,https://questiontheanswers.weebly.com/question...,True,19/08/21 01:15:36
298,SkullArcherx33,p6yn2q,,Foodforthought,Ask A Boss: ‘I Don’t Want to Be Like a Family ...,None,https://www.thecut.com/article/i-dont-want-to-...,True,18/08/21 19:45:04


In [154]:
submission_subreddit3_df = pd.DataFrame(list(getSubmission(300,'ChangeMyView')))
submission_subreddit3_df = addFieldsSub(submission_subreddit3_df)
submission_subreddit3_df

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,Virtual-Childhood-28,ubtqhu,I am a steadfast believer in pragmatic solutio...,changemyview,"CMV: As a Centrist Atheist in America, there a...",None,https://www.reddit.com/r/changemyview/comments...,True,25/04/22 19:50:16
1,Inaerius,ubtb8i,The saying goes that money can’t buy you happi...,changemyview,CMV: Money can buy you happiness.,None,https://www.reddit.com/r/changemyview/comments...,True,25/04/22 19:31:31
2,llort101,ubswev,I’m all for acceptance and inclusivity of gend...,changemyview,CMV: we shouldn’t glorify fat people.,None,https://www.reddit.com/r/changemyview/comments...,True,25/04/22 19:13:49
3,barbellseed,ubrsin,"I’m not really that religious, however, I do b...",changemyview,CMV: Believing in churches and any text associ...,None,https://www.reddit.com/r/changemyview/comments...,True,25/04/22 18:25:00
4,False-Seaworthiness7,ubrlec,I’ve heard a lot of bigger people complain tha...,changemyview,CMV: Amusement park rides should be built to f...,None,https://www.reddit.com/r/changemyview/comments...,True,25/04/22 18:16:09
...,...,...,...,...,...,...,...,...,...
268,hlanus,t5eigj,"Ever heard of ""Better Off Without 'Em"" by Chuc...",changemyview,CMV: Let Dixie Go,None,https://www.reddit.com/r/changemyview/comments...,True,03/03/22 00:29:35
269,burntoutibanker,t5e6nx,My view is that the geopolitical arena is domi...,changemyview,CMV: Realpolitik is the only correct framework...,None,https://www.reddit.com/r/changemyview/comments...,True,03/03/22 00:13:16
270,Master-namer-,t5casv,"Just like China, India has taken the stance of...",changemyview,CMV: India's stance on Russia-Ukriane war is r...,None,https://www.reddit.com/r/changemyview/comments...,True,02/03/22 22:42:52
271,TrickyBarrel,t5b210,Ukraine militarized itself despite agreements...,changemyview,CMV: Ukraine provoked Russia into war,None,https://www.reddit.com/r/changemyview/comments...,True,02/03/22 21:47:01


In [155]:
submission_subreddit4_df = pd.DataFrame(list(getSubmission(300,'TodayILearned')))
submission_subreddit4_df = addFieldsSub(submission_subreddit4_df)
submission_subreddit4_df

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,nathan_thinks,ubslwf,,todayilearned,TIL ‘Semantic Satiation’ is a psychological ph...,None,https://en.wikipedia.org/wiki/Semantic_satiati...,True,25/04/22 19:01:25
1,Austin63867,ubr230,,todayilearned,"TIL in 2009, 'Do You Realize' by The Flaming L...",None,https://pitchfork.com/news/50300-the-flaming-l...,True,25/04/22 17:52:32
2,GodhimselfUwU,u84ope,,todayilearned,TIL Philadelphia isn’t a state. Wtf.,None,https://en.wikipedia.org/wiki/Philadelphia,True,20/04/22 19:42:16
3,GaffersB,u1aqyc,,todayilearned,TIL that the most banned &amp; challenged book...,None,https://www.playgroundequipment.com/the-most-b...,True,11/04/22 15:47:41
4,ChronosBlitz,trtsi8,,todayilearned,TIL When Benjamin Franklin returned to America...,None,https://en.wikipedia.org/wiki/Benjamin_Franklin,True,30/03/22 00:19:18
...,...,...,...,...,...,...,...,...,...
291,vvavva69,t88v44,,todayilearned,TIL former American Idol judge Randy Jackson i...,None,https://www.nme.com/news/music/journey-announc...,True,06/03/22 21:26:52
292,Juslav,t87om7,,todayilearned,TIL In early 2000 there's a blind guy named Br...,None,https://www.nbcnews.com/id/wbna8731861,True,06/03/22 20:32:44
293,armando2311,t87ad7,,todayilearned,TIL the proportion of Iowa’s energy that comes...,None,http://iowaenvironmentalfocus.org/2021/04/12/t...,True,06/03/22 20:13:37
294,RexSueciae,t86zrf,,todayilearned,TIL that almonds and peaches are so geneticall...,None,http://www.suttonelms.org.uk/peach3.html,True,06/03/22 20:00:27


## Topic based submissions

In [156]:
political_submissions_df = pd.DataFrame(list(getSubmission(500, "Any","Joe Biden|Republican party|President")))
political_submissions_df = addFieldsSub(political_submissions_df, "Politics")
political_submissions_df   

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,spartachilles,w9s1q6,"Ever since the Second American Revolution, few...",Presidentialpoll,Social Democratic Convention of 1920 | A House...,Politics,https://www.reddit.com/r/Presidentialpoll/comm...,True,27/07/22 22:28:27
1,He_DidNothingWrong,ubt9p2,,interestingasfuck,"In France, on Election Day, it's tradition tha...",Politics,https://v.redd.it/3fdvhii68qv81,True,25/04/22 19:29:38
2,The_Chubby_Unicorn,ubt3f8,According to the Fox News exclusive:\n\n“The f...,DWAC_Stock,Trump to start Truthing within the next seven ...,Politics,https://www.reddit.com/r/DWAC_Stock/comments/u...,True,25/04/22 19:22:09
3,dniprshyrokyi,ubsbn8,,UkrainianConflict,"Aleksey Arestovich, adviser to the head of the...",Politics,https://socportal.info/en/news/an-expert-who-p...,True,25/04/22 18:49:09
4,jojenns,ubs23w,,boston,Former Boston police union president pleads gu...,Politics,https://www.wcvb.com/article/former-boston-pol...,True,25/04/22 18:37:05
...,...,...,...,...,...,...,...,...,...
488,ybenjira,tau9jl,"&amp;#x200B;\n\nBecause, if I'm El Khelaifi, t...",realmadrid,"Fans, rejoice, rejoice, you have no choice.",Politics,https://www.reddit.com/r/realmadrid/comments/t...,True,10/03/22 08:32:40
489,Lil_FiFi_,tau0c1,Tuition at CSULB is an accumulative collection...,CSULB,What do you want ASI to do for you as students?,Politics,https://www.reddit.com/r/CSULB/comments/tau0c1...,True,10/03/22 08:13:10
490,AutoModerator,tatto1,# Welcome to your home for everything [Pete](h...,Pete_Buttigieg,Home Base and Daily Discussion Thread (START H...,Politics,https://www.reddit.com/r/Pete_Buttigieg/commen...,True,10/03/22 08:00:10
491,hell_jumper9,tatoyp,,worldnews,President Rodrigo Duterte has expressed his wi...,Politics,https://www.gmanetwork.com/news/topstories/nat...,True,10/03/22 07:51:16


In [88]:
tech_submissions_df = pd.DataFrame(list(getSubmission(500, "Any","Artificial Intelligence|Cybersecurity|Internet|Gadgets|Facebook")))
tech_submissions_df = addFieldsSub(tech_submissions_df, "Tech")
tech_submissions_df  

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,url,topic,isSubmission,created_at
0,EnvironmentalEndUS,xbbpgo,,Glowsportingfeeds,best NFL live HD streams on the internet.&gt;&...,https://sports-live-daily.blogspot.com/p/nfl-l...,Tech,True,11/09/22 06:58:21
1,BloxedYT,x98ksd,"With the Queen passing, there's alot of disres...",teenagers,"No disrespect, but why is Gen Z so arrogant?",https://www.reddit.com/r/teenagers/comments/x9...,Tech,True,08/09/22 18:57:19
2,Forgetadapassword,x98aek,I remember the early years of the internet and...,conspiracy,The Queen is dead.,https://www.reddit.com/r/conspiracy/comments/x...,Tech,True,08/09/22 18:39:31
3,adz0r,wriqax,###LPL 2022 SUMMER \n[Official...,leagueoflegends,EDward Gaming vs. FunPlus Phoenix / LPL 2022 S...,https://www.reddit.com/r/leagueoflegends/comme...,Tech,True,18/08/22 12:48:38
4,adz0r,viw6io,###LCK 2022 SUMMER \n[Official...,leagueoflegends,T1 vs. Liiv SANDBOX / LCK 2022 Summer - Week 2...,https://www.reddit.com/r/leagueoflegends/comme...,Tech,True,23/06/22 12:48:58
...,...,...,...,...,...,...,...,...,...
491,SerendipitousWisdom,tbpplf,Any future updates will be posted as replies t...,u_SerendipitousWisdom,(March 11) Today's events in Ukraine and its r...,https://www.reddit.com/r/u_SerendipitousWisdom...,Tech,True,11/03/22 13:04:57
492,No-Occasion-9357,tbpmpo,Prior to my current boyfriend (possibly soon t...,dating,I can’t breathe… it’s likely my boyfriend of 2...,https://www.reddit.com/r/dating/comments/tbpmp...,Tech,True,11/03/22 13:00:44
493,event_threads,tbpmbj,Welcome to r/mma's discussion of ONE Champions...,MMA,[Official] ONE Championship: Lights Out - Live...,https://www.reddit.com/r/MMA/comments/tbpmbj/o...,Tech,True,11/03/22 13:00:12
494,wxman92,tbpkxl,,halo,Has anyone ever had lag like this before? I’ve...,https://v.redd.it/qx7rd5895rm81,Tech,True,11/03/22 12:58:14


In [157]:
env_submissions_df = pd.DataFrame(list(getSubmission(500, "Any","Biodegradable|Recycle|Climate Change|fossil fuel|forests")))
env_submissions_df = addFieldsSub(env_submissions_df, "Env")
env_submissions_df  

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,CephalonAhmes,xi70f6,[Source](https://www.testurl.com)\n\n**Update ...,scrappertest,submissiontitle,Env,https://www.reddit.com/r/scrappertest/comments...,True,19/09/22 09:25:04
1,CephalonAhmes,xc8io0,[Source](https://www.testurl.com)\n\n**Update ...,scrappertest,submissiontitle,Env,https://www.reddit.com/r/scrappertest/comments...,True,12/09/22 09:46:29
2,CephalonAhmes,xc8ijo,[Source](https://www.testurl.com)\n\n**Update ...,scrappertest,submissiontitle,Env,https://www.reddit.com/r/scrappertest/comments...,True,12/09/22 09:46:18
3,CephalonAhmes,x6iikc,[Source](https://www.testurl.com)\n\n**Update ...,scrappertest,submissiontitle,Env,https://www.reddit.com/r/scrappertest/comments...,True,05/09/22 15:10:30
4,CephalonAhmes,x0kfpo,[Source](https://www.testurl.com)\n\n**Update ...,scrappertest,submissiontitle,Env,https://www.reddit.com/r/scrappertest/comments...,True,29/08/22 09:34:59
...,...,...,...,...,...,...,...,...,...
494,NecrostusXerxemon,sy5ana,I think a lot of Korra detractors put a lot of...,legendofkorra,Why I Don't Hate The Mech,Env,https://www.reddit.com/r/legendofkorra/comment...,True,21/02/22 21:40:25
495,CantGetNoSleep88,sy55r2,I've yet to find a pair that really suit my gr...,gravelcycling,Which sunglasses do you use for gravel?,Env,https://www.reddit.com/r/gravelcycling/comment...,True,21/02/22 21:34:52
496,cynicalmermaid,sy4ktc,**…We interrupt this program to bring you an ...,lesbianr4r,31/F4F/USA – BREAKING NEWS: IT’S ANOTHER LONEL...,Env,https://www.reddit.com/r/lesbianr4r/comments/s...,True,21/02/22 21:11:24
497,listtowardslight,sy3z6o,Rather than debate one single approach to comb...,climatechange,Why fighting climate change has to be a many-p...,Env,https://www.reddit.com/r/climatechange/comment...,True,21/02/22 20:47:34


In [90]:
health_submissions_df = pd.DataFrame(list(getSubmission(500, "Any","healthcare|medical|hospital|dermatologist|surgery")))
health_submissions_df = addFieldsSub(health_submissions_df, "Health")
health_submissions_df  

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,url,topic,isSubmission,created_at
0,wedditmoderator,x2xoyw,# About\n\nThis megathread is for any and all ...,weddingplanning,Monthly Megathread for COVID-19 - September 2022,https://www.reddit.com/r/weddingplanning/comme...,Health,True,01/09/22 04:01:10
1,wedditmoderator,wd7oq4,# About\n\nThis megathread is for any and all ...,weddingplanning,Monthly Megathread for COVID-19 - August 2022,https://www.reddit.com/r/weddingplanning/comme...,Health,True,01/08/22 04:01:09
2,dangersandwich,vox61q,# Intro\n\nWelcome to the AskEngineers quarter...,AskEngineers,The Q3 2022 AskEngineers Salary Survey,https://www.reddit.com/r/AskEngineers/comments...,Health,True,01/07/22 10:00:11
3,wedditmoderator,vorgen,# About\n\nThis megathread is for any and all ...,weddingplanning,Monthly Megathread for COVID-19 - July 2022,https://www.reddit.com/r/weddingplanning/comme...,Health,True,01/07/22 04:01:11
4,wedditmoderator,v285s5,# About\n\nThis megathread is for any and all ...,weddingplanning,Monthly Megathread for COVID-19 - June 2022,https://www.reddit.com/r/weddingplanning/comme...,Health,True,01/06/22 04:01:09
...,...,...,...,...,...,...,...,...,...
489,xpurpleslurpee,tbqboz,Really could’ve been less than 7 weeks but I s...,step1,Underdog Story 165–&gt;pass in 7 weeks,https://www.reddit.com/r/step1/comments/tbqboz...,Health,True,11/03/22 13:38:22
490,ssajan1,tbqbj7,I’m basing this question on a 2005 study that ...,atheism,What do y’all think is the reason or even reas...,https://www.reddit.com/r/atheism/comments/tbqb...,Health,True,11/03/22 13:38:07
491,chan___kun,tbqb6v,"Like, i just wanna be able to wear cool sungla...",teenagers,should i get lazer eye surgery?,https://www.reddit.com/r/teenagers/comments/tb...,Health,True,11/03/22 13:37:31
492,Lilip4,tbqaph,,Skincare_Addiction,Hello there:) I have sensitive skin.. and the ...,https://www.reddit.com/gallery/tbqaph,Health,True,11/03/22 13:36:51


In [158]:
edu_submissions_df = pd.DataFrame(list(getSubmission(500, "Any","education|scholarship|grades|curriculum|professors")))
edu_submissions_df = addFieldsSub(edu_submissions_df, "Education")
edu_submissions_df  

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,spartachilles,w9s1q6,"Ever since the Second American Revolution, few...",Presidentialpoll,Social Democratic Convention of 1920 | A House...,Education,https://www.reddit.com/r/Presidentialpoll/comm...,True,27/07/22 22:28:27
1,PurrPrinThom,uq2pft,"To best help find solutions to your query, ple...",AskProfessors,Frequently Asked Questions,Education,https://www.reddit.com/r/AskProfessors/comment...,True,15/05/22 09:48:58
2,Orrion_the_Kitsune_,ubtdqb,"As it stands, the consensus in the scientific ...",flatearth,I'm a scholar for physics (among other things....,Education,https://www.reddit.com/r/flatearth/comments/ub...,True,25/04/22 19:34:42
3,SpeakTruthPlease,ubsf8a,,conspiracy,Reddit ❤️ Education,Education,https://i.redd.it/r2fuiq5v1qv81.png,True,25/04/22 18:53:30
4,fightfreeNJ,ubs296,Warning: This will be a lengthy post but I hop...,conspiracy,We Are Approaching a Soviet Union-Style Social...,Education,https://www.reddit.com/r/conspiracy/comments/u...,True,25/04/22 18:37:16
...,...,...,...,...,...,...,...,...,...
491,Business_Genius,tb1zs1,Hi Everyone. I am from Bangladesh doing Cambri...,IntltoUSA,How rigorous is the CAIE A level program for c...,Education,https://www.reddit.com/r/IntltoUSA/comments/tb...,True,10/03/22 16:05:46
492,valicheriyuka,tb1yxw,The word sex was a taboo in my family and to h...,Kerala,Sex Education resources,Education,https://www.reddit.com/r/Kerala/comments/tb1yx...,True,10/03/22 16:04:40
493,NurseGryffinPuff,tb1x9m,I had two recent interesting conversations wit...,nursing,What are lies you tell patients about yourself...,Education,https://www.reddit.com/r/nursing/comments/tb1x...,True,10/03/22 16:02:36
494,notdoomscrolling,tb1vz8,"If the latter, will it hurt my chances of tran...",ApplyingToCollege,Should I go to a respected but expensive colle...,Education,https://www.reddit.com/r/ApplyingToCollege/com...,True,10/03/22 16:01:04


## Fetching Comments

In [159]:
submission_subreddit1_df

,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,Addy_Bishop,ubrebd,I understand what stock is but who creates it?...,explainlikeimfive,ELI5 where does stock come from?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,25/04/22 18:07:22
1,Ambitious_Category85,trsva6,,explainlikeimfive,ELI5: How do seasons exists? Isn't the earth's...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 23:31:10
2,twerkinmidget,trs358,"Switching the clocks seems excessive, it IS th...",explainlikeimfive,ELI5: I don’t understand the point of time zon...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:52:06
3,AlienRouge,trrkza,,explainlikeimfive,ELI5: what is Roko’s Basilisk?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:31:22
4,Not-Alpharious,trpcdk,Wouldn’t it be more of a survival advantage to...,explainlikeimfive,ELI5: Why do animals have mating seasons inste...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:07:51
...,...,...,...,...,...,...,...,...,...
281,reapingsulls123,t8kxc7,Yes I know you have to legally get out of thei...,explainlikeimfive,ELI5: How emergency services manage to get to ...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 08:49:53
282,EazyAB,t8krqd,What exactly makes an atomic bomb so catastrop...,explainlikeimfive,ELI5: Atomic Bombs?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 08:38:17
283,beepingview,t8jfpd,"1. If you buy a smart TV, can you watch both N...",explainlikeimfive,Eli5: Can you watch both Network and Cable on ...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 07:04:35
284,Patdick-Funny,t8isyt,,explainlikeimfive,Eli5: What causes babies/children able to lear...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,07/03/22 06:21:02


In [93]:
comments_lst = []
comments_lst.extend(api.search_comments(link_id='ubrebd', subreddit=['ExplainLikeImFive'],
                                            filter=['id','parent_id','permalink','author', 'title', 
                                                    'subreddit','body','num_comments','score'],
                                            bodynot = "[removed]",
                                            limit=2     
                                            ) )
comments_df = pd.DataFrame(comments_lst)

d:\Applications\JupyterFiles\envs\R\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [94]:
comments_df

,author,body,created_utc,id,parent_id,permalink,score,subreddit,created,d_
0,Addy_Bishop,I meant the warren buffet level stocks. I was...,1652069249,i7vqq7u,t1_i7vq8qp,/r/explainlikeimfive/comments/ubrebd/eli5_wher...,1,explainlikeimfive,1.652084e+09,"{'author': 'Addy_Bishop', 'body': 'I meant the..."
1,kxleepy,what did u mean by ultra rich things? stocks? ...,1652068956,i7vq8qp,t1_i7vq3lh,/r/explainlikeimfive/comments/ubrebd/eli5_wher...,1,explainlikeimfive,1.652083e+09,"{'author': 'kxleepy', 'body': 'what did u mean..."


In [101]:
y = int(len(submission_subreddit1_df))
y

286

In [103]:
comments_lst1 = []
for i in range(286):
    comments_lst1.extend(api.search_comments(link_id=submission_subreddit1_df.iloc[i]['id'], subreddit=submission_subreddit1_df.iloc[i]["subreddit"],
                                            filter=['id','parent_id','permalink','author', 'title', 
                                                    'subreddit','body','num_comments','score'],
                                            bodynot = "[removed]",
                                            limit=2     
                                            ))

comments1_df = pd.DataFrame(comments_lst1)
comments1_df

,author,body,created_utc,id,parent_id,permalink,score,subreddit,created,d_
0,Addy_Bishop,I meant the warren buffet level stocks. I was...,1652069249,i7vqq7u,t1_i7vq8qp,/r/explainlikeimfive/comments/ubrebd/eli5_wher...,1,explainlikeimfive,1.652084e+09,"{'author': 'Addy_Bishop', 'body': 'I meant the..."
1,kxleepy,what did u mean by ultra rich things? stocks? ...,1652068956,i7vq8qp,t1_i7vq3lh,/r/explainlikeimfive/comments/ubrebd/eli5_wher...,1,explainlikeimfive,1.652083e+09,"{'author': 'kxleepy', 'body': 'what did u mean..."
2,Way2Foxy,Effectively no difference. The distance ranges...,1648615688,i2ofb6y,t1_i2ngssw,/r/explainlikeimfive/comments/trsva6/eli5_how_...,1,explainlikeimfive,1.648630e+09,"{'author': 'Way2Foxy', 'body': 'Effectively no..."
3,ntengineer,**Please read this entire message**\r\n\r\n---...,1648610861,i2o6q5m,t3_trsva6,/r/explainlikeimfive/comments/trsva6/eli5_how_...,1,explainlikeimfive,1.648625e+09,"{'author': 'ntengineer', 'body': '**Please rea..."
4,newytag,Wow what a fucking stupid response. Don't be a...,1648703027,i2t8mnc,t1_i2t20qc,/r/explainlikeimfive/comments/trs358/eli5_i_do...,1,explainlikeimfive,1.648717e+09,"{'author': 'newytag', 'body': 'Wow what a fuck..."
...,...,...,...,...,...,...,...,...,...,...
567,GangOfNone,Depends on the make of the TV. Most have Netfl...,1646669330,hzptvfc,t1_hzohhj5,/r/explainlikeimfive/comments/t8jfpd/eli5_can_...,1,explainlikeimfive,1.646684e+09,"{'author': 'GangOfNone', 'body': 'Depends on t..."
568,hansolo8097,I'd have given him or her an award if I could....,1646759743,hzuyid9,t1_hzpprtt,/r/explainlikeimfive/comments/t8isyt/eli5_what...,1,explainlikeimfive,1.646774e+09,"{'author': 'hansolo8097', 'body': 'I'd have gi..."
569,activelyresting,"Additionally, babies are hearing their native ...",1646681165,hzqoggi,t3_t8isyt,/r/explainlikeimfive/comments/t8isyt/eli5_what...,1,explainlikeimfive,1.646696e+09,"{'author': 'activelyresting', 'body': 'Additio..."
570,LeoRidesHisBike,"""Anus"" is definitely more specific than ""ass""....",1646681535,hzqpete,t1_hzozgjb,/r/explainlikeimfive/comments/t8ipem/eli5_why_...,3,explainlikeimfive,1.646696e+09,"{'author': 'LeoRidesHisBike', 'body': '""Anus"" ..."


In [131]:
coms = []
for index in range(len(comments1_df)):
    if (("[removed]" == comments1_df["body"].iloc[index]) or ("[deleted]" == comments1_df["body"].iloc[index]) or
        ("t3_" not in comments1_df["parent_id"].iloc[index])):
        coms.append(index)#same for parent_body
          # consider these comments
            
#     else:
#           # dont consider these as these fall in 5% bracket.
#         coms.append([row['author'],row['body'],row['created_utc'],row['id'],row['parent_id'],row['permalink'],row['score'],row['subreddit'],row['created'],row['d_']])
comments1_df.drop(labels=coms, axis=0, inplace=True)
comments1_df_final = comments1_df.reset_index(drop=True)

In [132]:
comments1_df_final_2 = addFieldsCom(comments1_df_final, submission_subreddit1_df )
comments1_df_final_2

<class 'pandas.core.frame.DataFrame'>


C:\Users\Heet\AppData\Local\Temp\ipykernel_26172\1800867481.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['parent_body'].iloc[index] = str(df1['selftext'].loc[df1['id']==key])


,author,body,id,parent_id,subreddit,parent_body,created_at,full_link
0,ntengineer,**Please read this entire message**\r\n\r\n---...,i2o6q5m,t3_trsva6,explainlikeimfive,"1 \nName: selftext, dtype: object",30/03/22 03:27:41,https//:reddit.com/r/explainlikeimfive/comment...
1,FellowConspirator,"No. What KI pills do is prevent your thyroid, ...",i2nxfia,t3_tro1xl,explainlikeimfive,"5 \nName: selftext, dtype: object",30/03/22 02:11:16,https//:reddit.com/r/explainlikeimfive/comment...
2,Flair_Helper,**Please read this entire message**\n\nYour su...,i2qacro,t3_trdonh,explainlikeimfive,"7 \nName: selftext, dtype: object",30/03/22 16:05:34,https//:reddit.com/r/explainlikeimfive/comment...
3,Reggin-RBB3,Most answers miss the biggest factor: if all y...,i2oe2cn,t3_tra91z,explainlikeimfive,9 Is there a reason? Is the meat nasty as t...,30/03/22 04:35:42,https//:reddit.com/r/explainlikeimfive/comment...
4,goodlordzenmaster,Web3: It's the next iteration of the internet....,imbbffp,t3_tr6dl4,explainlikeimfive,"11 \nName: selftext, dtype: object",29/08/22 22:33:08,https//:reddit.com/r/explainlikeimfive/comment...
...,...,...,...,...,...,...,...,...
160,SidewaysWild,"Atleast here in Ontario, our emergency vehicle...",hzsp75g,t3_t8kxc7,explainlikeimfive,281 Yes I know you have to legally get out ...,08/03/22 04:03:53,https//:reddit.com/r/explainlikeimfive/comment...
161,ElfMage83,Rule 7 requires that you search the sub before...,hzp2kih,t3_t8krqd,explainlikeimfive,282 What exactly makes an atomic bomb so ca...,07/03/22 12:28:57,https//:reddit.com/r/explainlikeimfive/comment...
162,Flair_Helper,**Please read this entire message**\n\nYour su...,hzp2ju2,t3_t8krqd,explainlikeimfive,282 What exactly makes an atomic bomb so ca...,07/03/22 12:28:45,https//:reddit.com/r/explainlikeimfive/comment...
163,activelyresting,"Additionally, babies are hearing their native ...",hzqoggi,t3_t8isyt,explainlikeimfive,"284 \nName: selftext, dtype: object",07/03/22 19:26:05,https//:reddit.com/r/explainlikeimfive/comment...


In [160]:
final_submissions_df = pd.concat([submission_subreddit1_df,submission_subreddit2_df,submission_subreddit3_df,submission_subreddit4_df, political_submissions_df,health_submissions_df, tech_submissions_df,edu_submissions_df, env_submissions_df])
final_submissions_df

,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at,url
0,Addy_Bishop,ubrebd,I understand what stock is but who creates it?...,explainlikeimfive,ELI5 where does stock come from?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,25/04/22 18:07:22,NaN
1,Ambitious_Category85,trsva6,,explainlikeimfive,ELI5: How do seasons exists? Isn't the earth's...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 23:31:10,NaN
2,twerkinmidget,trs358,"Switching the clocks seems excessive, it IS th...",explainlikeimfive,ELI5: I don’t understand the point of time zon...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:52:06,NaN
3,AlienRouge,trrkza,,explainlikeimfive,ELI5: what is Roko’s Basilisk?,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:31:22,NaN
4,Not-Alpharious,trpcdk,Wouldn’t it be more of a survival advantage to...,explainlikeimfive,ELI5: Why do animals have mating seasons inste...,None,https://www.reddit.com/r/explainlikeimfive/com...,True,29/03/22 22:07:51,NaN
...,...,...,...,...,...,...,...,...,...,...
494,NecrostusXerxemon,sy5ana,I think a lot of Korra detractors put a lot of...,legendofkorra,Why I Don't Hate The Mech,Env,https://www.reddit.com/r/legendofkorra/comment...,True,21/02/22 21:40:25,NaN
495,CantGetNoSleep88,sy55r2,I've yet to find a pair that really suit my gr...,gravelcycling,Which sunglasses do you use for gravel?,Env,https://www.reddit.com/r/gravelcycling/comment...,True,21/02/22 21:34:52,NaN
496,cynicalmermaid,sy4ktc,**…We interrupt this program to bring you an ...,lesbianr4r,31/F4F/USA – BREAKING NEWS: IT’S ANOTHER LONEL...,Env,https://www.reddit.com/r/lesbianr4r/comments/s...,True,21/02/22 21:11:24,NaN
497,listtowardslight,sy3z6o,Rather than debate one single approach to comb...,climatechange,Why fighting climate change has to be a many-p...,Env,https://www.reddit.com/r/climatechange/comment...,True,21/02/22 20:47:34,NaN


In [161]:
import json

In [162]:
xyz = json.loads(final_submissions_df.to_json(orient = "records"))
#del xyz['data']['index']
json_object= json.dumps(xyz)
#parsed = json.loads(xyz)
#json_object = json.dumps(parsed, indent=4) 

with open("sample2.json", "w") as outfile:
    outfile.write(json_object)